In [34]:
from openai import AzureOpenAI
from openai.types import Embedding
from dotenv import load_dotenv
import pytest
import os
import logging
import tiktoken 
from json import JSONDecodeError
from sklearn.metrics.pairwise import cosine_similarity
from cluestar import plot_text
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import ast

load_dotenv('.env.shared')
load_dotenv('.env.secret')

True

In [35]:
openai_client = AzureOpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_API_BASE'),
    api_version=os.getenv('OPENAI_API_VERSION')
)


In [37]:
user_prompt = lambda a, b: """
WELLBORE NAME: """+a+""" 

PARAGRAPH: """+b

system_prompt = """The user will provide you with a paragraph of text and a metadata string.
The data will follow this format:

WELLBORE NAME: 
PARAGRAPH:

The paragraphs and metadata can be in different languages. Always answer in english. They all come from the same domain of oil and gas exploration and production. 

You will create questions that can be answered from the text, alongside the two answers for each question.
The first answer should be as short and explicit as possible. Put extra weight on making it as short as possible with no filler words at all. Use the exact words and phrasing from the text.
The second answer should be answered using natural language.
The wellbore name should always be explicitly written in the questions and explicitly written in the second answer.
You may also use exact sentences or shorten the sentences to phrases to generate examples.
You may paraphrase the content in sentences or change them slightly but you must preserve both language and style of individual texts.
If there are no examples to generate then create an empty list. 
Provide a confidence score between 0 and 1 for each of the question answer pairs where 1 is full confidence in the answer, and 0 is no confidence in the answer.

You are to produce a JSON formatted output, with JSON output only in the following form:

[{"Q": "<Question 1a>", "A1": "<Answer 1b>", "A2": "<Answer 1c>", "C" : "<Confidence 1d>"}, 
 {"Q": "<Sentence 2a>", "A1": "<Sentence 2b>", "A2": "<Answer 2c>", "C" : "<Confidence 2d>"},
 ...,
 {"Q": "<Sentence Na>", "A1": "<Sentence Nb>", "A2": "<Answer Nc>", "C" : "<Confidence Nd>"}]

You are to determine yourself how many examples N you can come up with, but make as many as possible from the text provided, preferably at least 10. It is very important to make sure its always JSON formatted.
"""

In [27]:
# user_prompt = lambda a, b: """
# WELLBORE NAME: """+a+""" 

# PARAGRAPH: """+b 

# data_list = [
# {
# "metadata" : "15/5-1",
# "paragraph" : 'IV PETROPHYSICS AND WELL PRODUCTIVITY IV. 1. Loq Interpretation Investigation of the logs in well 15/5-1 showed hydro- carbon sand in the interval 3558-3614 RKBx, corre- sponding to total thickness of 56 m. No hydrocarbon water contact was identified.  The sand section may be subdivided into four separate pay zones, each zone being separated by thin impermeable layers. Pay zone Pay interval Thickness Avg. porosity Avg. SW m RKB 1 3558.6-3590.9 15 11 2 3594.8-3597.9 12 25 4 3608.8-3613.7 12 22 Total 14 14 IV. Observations The upper zone is the main zone, equivalent to 83% of the oil column. The cut-off criteria used in the log evaluation were: x The CP1 log is used as depth reference. RKB elevation ,.2-5m Refer Ficrui-e IV.1 and 7eservoir Fvaluation Display Section 8.'
# },
# {
# "metadata" :  '30/7-7',
# "paragraph" : 'The clay calculation from MN plot is excluded because this gives zero percent clay in some interval and this does not fit to any of the other clay indicatores. These clay intervals became hydrocarbon bearing with MN as clay indicator. These intervals Below 4975 the sand become more shaly. From the Gamma ray the clean lin (GR min) seems to be between 15 and 41 API. If the 41 API is used the gamma ray as clay indicator will dominate in this interval, because of this 15 API used like the zone above. 2.3 HYDROCARBON DENSITY The hydrocarbon density is set ot even if no or almost no hydrocarbon effect can be seen from the logs. The probability of having heavy oil at this high pressure and temperature is very low. With following effect no high hydrocarbon effect on the -logs could be expected: High water saturation, hydrocarbon fluid of volatile oil or condensate and possibly deep invation. 2.4 CUT orF VALUE Cut off porosity 10% Cut off clay 40% Cut off Sw 70% The Sw cut off value uf 70% is due to the fact that most probable the hydrocarbon fluid is gas or gas condensate. If hydrocarbon is oil the cut off values should be lower. With heavy oil the possibility of very little movable oil is present. By using these cut offs high frequency of all three cut offs criteria acting at same intervals are observed. This tendency might confirm the validity of the cut offs.'
# },
# {"metadata" : '25/4-1',
# "paragraph" : '''FORMATION TESTER S.P.E. PL 28 Is DAIE 19/10/72 OPE RA10 St art Ended R P00 DURAND '014. . 6E1 total 11. of 01 Company Witness CARRF 26/14-1 WELL 'FORMATION JIJRASSIQUC GENERAL INFORMATIONS MUD (Drilling Mud) Density Viscosity 50 InfiPPEKA completion letIdSoi in, 39 Source of it trate Hole diameter Jurassic R eleIVOir Reason for test ing FOR MAftObi evpi Casing  Level  SAMPLINd Late wiper trip Carts' tonnidgy Sandstone Granulornetry  Porosity ______________17 II Permeability .dims Salinity of interstitial water Static Reservoir Pre ssure Head of mud RwP-fl 1flnc .04 of test Testing depth 3504,6 absolute depth -34 81 OVERALL RESULTS r.Ace C.ASCII.DIF TOOLS Combo FIT 1 ool FIT or FIT _______ Mode of placement CM. Diameter of char ges, 1/8 kind of Setting I Pe or pad HARD Sompiing chamber 110 or ./2 21 3/4 Hydraulic circuit reguiat Or TEST Pressure /COI IL Pad Weisser& psi a shut in Hydrostatic pr...... reeding prinstire cifrapolatied si Pr lessor./ ft owing Start 7200 End. /600 Flowing ti rne Sampling chamber till up rot, ___________ 70 % BHP. reading m2 extrapolated Sampling chamber pressure reading at surrace .4acto. Permeability based on pressure calcoJations ________________________ from build up pressure curve 0  10000 _1_ GA RESULTS 75.2 cf 4. gasoline 500 CC Is' It WATER. FILTRATE- FILIRAIE 2000 CC Rent 24 16 'IC DRY EVENTUAL FAILURE kg m2 kg/cm2 _ duration 1_111S1 Observation5.____________ FYI tranafe. CHsemenIs ELF NORGE'''
# }
# ]
# # metadata = '34/10-2'
# # paragraph = 'Litologi Sedimentbeskrivelse basert pa daglig telex fra Statfjord. Dyp tekst refererer til KB. De Overste ca 500 metrene av brOnnen bestar av sand og kan trolig henfOres til Pleistocene. Fra 5-600 til ca 900 bestar sedimentene hovedsakelig av leire og leirstein med sandlag. Tuffrik skifer er telex rapportert fra 915 dyp, men mangefargete leirsteiner: (varicoloured claystones) er be- skrevet fra 700 m, slik at tuffsonen muligens ligger grunnere enn ca 900 m. intervallet 790 - 850 kan det ut fra GR/Sonic logg muligens antydes en tuffsone, men fra dette intervallet er tuff ikke rapportert. Balder er derfor antydet innenfor intervallet 790 - ca 000 m. Fra ca 000 til 878 bestar sedimentene av gra leirstein med lag av gra og brunlig kalksten og enkelte sandstenslag. Ut fra loggene virker hele denne sekvensen noksa monoton, og den antas representere Shetland gruppen. Fra 878 til 885 viser GR-loggen en klar hot-shale, mens sedimentene dette intervallet er beskrevet som en rOd leirsten ned til 940 der det er et loggebrudd. Sekven- sen fra topp hot shale til loggebruddet pa 940 anses for & tilhOre Humber gruppen Intervallet fra 940 til 125 med interbedded kull, sandsten, siltsten og skifer representerer Brent-Formasjonen. En markert overgang pa GR-log pa 125 til en relativt homogen skifersekvens markerer overgangen til Dunlin Forma- sjonen. Den er beskrevet som sandsten og siltsten Ovre del av formasjonen med gradvis overgang til en homogen sek- yens med mOrk gra skifer. Overgangen til den mer sandige Statfjord-Formasjonen viser ved tydelige utslag pa GR og Sonic logger pa ca 300 m. Statfjord-Formasjonen bestar av sandsten og gra skifer interbedded. Bunnen av Statfjord er noe usikker, men er her satt til dyp 540 som SLA/JaV-notat am 34/10-1 og 2.'

# metadata = '15/5-1'
# paragraph = 'A piled steel jacket of this sort is probably not the most cost effective fixed structure for this location. Section 1.1.5 deals with the alternatives that exist and from this superficial examination, it is probable that the single leg jack-up concept, with pre-commissioned topside equipment installed over predrilled template would be comparable in cost. a. Influence of Method of Installation Barge launching the structure on location is the conventional method of installation, followed by controlled flooding to up- right the structure and final sinking to the seabed. If such a method is adopted, at least one face of the structure must have two parallel lauch tracks, and strong launch truss must be incorporated internally. This implies that at least legs must be used arranged in two rows of 4. The major disadvantage of such an eight-leg structure is that the wave force attracted are larger than for more open four legged structure . This is only true, however, if provision for installation is not considered and for heavier structures. The conventional North Sea approach has been to build large self-floating legs, which attract considerable extra wave force, (e.g. Thistle, Ninian, Magnus) or fabricate purpose built floa- taion rafts, which can be submerged with the structure atta- ched (e.g. Froties). The Forties system is not now favoured because of the lack of resuability of the floatation rafts and in each case, considerable extra problems of fabrication are introduced. The most complex and time consuming fabrication for self-floating jackets is for the floatation legs on the ground,,and this must be substantially complete before the main jacket structure can be built. This.complicatps and extends the fabrication process. A four legged structure without provision for installation will generally weight less because of the reduced wave forces and more open structure. It is not as redundant as an eight legged'


# prompt = user_prompt(metadata, paragraph)

In [87]:
# for dict in data_dict:
#     print(user_prompt(dict["metadata"], dict["paragraph"]))


METADATA: 15/5-1 

PARAGRAPH: IV PETROPHYSICS AND WELL PRODUCTIVITY IV. 1. Loq Interpretation Investigation of the logs in well 15/5-1 showed hydro- carbon sand in the interval 3558-3614 RKBx, corre- sponding to total thickness of 56 m. No hydrocarbon water contact was identified.  The sand section may be subdivided into four separate pay zones, each zone being separated by thin impermeable layers. Pay zone Pay interval Thickness Avg. porosity Avg. SW m RKB 1 3558.6-3590.9 15 11 2 3594.8-3597.9 12 25 4 3608.8-3613.7 12 22 Total 14 14 IV. Observations The upper zone is the main zone, equivalent to 83% of the oil column. The cut-off criteria used in the log evaluation were: x The CP1 log is used as depth reference. RKB elevation ,.2-5m Refer Ficrui-e IV.1 and 7eservoir Fvaluation Display Section 8.

METADATA: 30/7-7 

PARAGRAPH: The clay calculation from MN plot is excluded because this gives zero percent clay in some interval and this does not fit to any of the other clay indicatores. 

In [61]:
# Load the shared environment variables, not secrets




# completion = openai_client.chat.completions.create(model=os.environ["GPT432k_DEPLOYMENT"], 
#                                             messages=[{"role": "system", 
#                                                        "content": system_prompt}, 
#                                                        {"role": "user", "content": prompt}])

In [81]:
# print(completion.choices[0].message.content)

[{"Q": "What does section 1.1.5 of the 15/5-1 metadata deal with?", "A1": "Alternatives that exist for the cost effective fixed structure for this location.", "A2": "Section 1.1.5 of the metadata 15/5-1, addresses the alternatives that are there which could be optimal and cost effective for the fixed structure in this location.", "C" : "1"}, 

 {"Q": "What concept does the 15/5-1 metadata propose as comparable in cost to a piled steel jacket?", "A1": "Single leg jack-up concept, with pre-commissioned topside equipment installed over predrilled template.", "A2": "According to the metadata 15/5-1, the single leg jack-up idea wherein pre-commissioned topside equipment is installed over the predrilled template is proposed to be similar in cost to a piled steel jacket.", "C" : "0.9"},

 {"Q": "What is the conventional method of installation as per the paragraph on 15/5-1 metadata?", "A1": "Barge launching the structure on location, followed by controlled flooding to up- right the structure 

In [54]:
data_link = '../data/annotates good text'
# number_of_sentences = 5
# data_df = pd.read_csv(f"../data/passages_discovery_evaluation_report_{number_of_sentences}_sentences.csv")
data_df = pd.read_csv(f"{data_link}.csv")
# data_df = data_df[0:3]

In [44]:
print(data_df)

   Passage_id                                       _id  \
0           1  fbb20433dc987ebc3006d70f39836b6f3aeb9302   
1           2  fbb20433dc987ebc3006d70f39836b6f3aeb9302   
2           3  fbb20433dc987ebc3006d70f39836b6f3aeb9302   

                                            filename  page  \
0  25_8-10 S__DRILLING__DRILLING_CEMENT_1997-08-3...     3   
1  25_8-10 S__DRILLING__DRILLING_CEMENT_1997-08-3...     4   
2  25_8-10 S__DRILLING__DRILLING_CEMENT_1997-08-3...     5   

                                             content possible_language  \
0  ESS Introduction The job reports issued by our...                en   
1  Plug across 13 3/8 shoe Well Report.. Cementin...                en   
2  Plug and Abandonment of the Well: The well was...                en   

                langdetect  content_could_be_natural_language  \
0  [en:0.9999980283917008]                               True   
1   [en:0.999996245817104]                               True   
2  [en:0.9999960425313

In [55]:
data_list = [{"prompt":user_prompt(row["WELLBORE NAME"], str(row["Paragraph"])), "Passage_id":row["Passage_id"]} for index, row in data_df.iterrows()]
print(data_list)

[{'prompt': "\nWELLBORE NAME: 25/8-10 S \n\nPARAGRAPH: ESS Introduction The job reports issued by our offshore personnel along with computer printouts of the jobs are filed at Halliburton's office in Tananger and can be supplied if required. Job Summary 32 30 conductor casing The 36 hole was drilled to TD at 229 MD. The conductor was cemented from the shoe at 229 MD and back to the mudline at 154m MD. Prior to the job there was discussion on compressive strength development of conductor cement slurries. Several options were discussed, but the job was conducted with single 15.80ppg cement slurry. 200% excess volume was used. The cement slurry was displaced with 55bbls seawater at an average displacement rate of 5.7bpm. No problems were reported during the job. 20 13 3/8 casing A 17 1/2 hole was drilled to 1091 MD. The 20 13 3/8 casing was run to 1076 m MD and cemented by pumping 703bbls 13.2ppg lead slurry, and 159bbls 15.80ppg tail slurry. 100 % excess on open hole was used for volume 

In [57]:
# load_dotenv("../.env.shared")
# load_dotenv("../.env.secret")


# openai_client = AzureOpenAI(
#     api_version=os.environ["OPENAI_API_VERSION"],
#     azure_endpoint=os.environ["OPENAI_API_BASE"],
#     api_key=os.environ["OPENAI_API_KEY"]
# )




# data_list = [{"metadata":row["WELLBORE NAME"], "paragraph":row["passage"]} for index, row in data_df.iterrows()]


# def ask_question(data):
#     prompt = user_prompt(data["metadata"], data["paragraph"])
#     # system_prompt = data["metadata"]
#     response = openai_client.chat.completions.create(
#         model=os.environ["GPT432k_DEPLOYMENT"], 
#         messages=[
#             {"role": "system", "content": system_prompt}, 
#             {"role": "user", "content": prompt}
#         ]
#     )
#     return response.choices[0].message.content

# def main():
#     error_list = []
#     with ThreadPoolExecutor() as executor:
#         responses = list(executor.map(ask_question, data_list))
#         print(responses)
#         # for list_dict in responses:
#         #     try:
#         #         reformed = ast.literal_eval(list_dict)
#         #         df1 = pd.DataFrame(reformed)
#         #         df = pd.concat([df, df1])
#         #     except:
#         #         print("Not proper output")
#         #         error_list.append(list_dict)
        
#     return [responses, error_list]

# main_run = main()
# responses = main_run[0]
# error_list = main_run[1]



['[]\n', '[{"Q": "What was the reason for testing in the 25/2-2 metadata?", "A1": "s_aample for PVT Analysis.", "A2": "The reason for testing per the metadata 25/2-2 was to get the s_aample for PVT Analysis.", "C" : "0.9"},\n {"Q": "What type of formation is mentioned in the text for the 25/2-2 metadata?", "A1": "EocenQ", "A2": "The formation mentioned in the text, related to the metadata 25/2-2, was the EocenQ.", "C" : "0.8"},\n {"Q": "What is the porosity value given in the 25/2-2 metadata text?", "A1": "29 0/0", "A2": "The porosity value given in the text of metadata 25/2-2 is \'29 0/0\'.", "C" : "0.95"},\n {"Q": "What is the state of permeability in the 25/2-2 metadata text?", "A1": "very good", "A2": "The permeability in the text associated with metadata 25/2-2 is described as \'very good\'.", "C" : "0.9"},\n {"Q": "What is the salinity of interstitial water in the 25/2-2 metadata?", "A1": "53 g/1", "A2": "In the metadata 25/2-2 text, the salinity of interstitial water is 53 g/1."

In [115]:
# print(data_list)

[{'prompt': "\nWELLBORE NAME: 25/2-2 \n\nPARAGRAPH: V FORMATION TESTER S.P.E. PL N5 Ended O.O.H.F.' 211-130 of R.I.H. TLLOUS OPERATOR  Witness: PINET WELL: 25/2-2 GENERAL INFORMATIONS Density  1,20 Viscosity : Free water  5,0 Salinity : MUD 40 cp 37_,V1 Source of fit Irate Press liner Hole diameter : 81 Cake ________ Late wiper 2H Reservoir Fr gg _Member trip ?", 'passage_id': 0}]


In [52]:
# print(responses)

[]


In [36]:
# df2 = [{'Q': 'What investigation was conducted in well 15/5-1?', 'A1': 'Investigation of the logs'}, {'Q': 'What', 'A1': 'Investigation'}]

# df1 = [{"A" : "OK", "B": "OK2"}]

# df3 = [{"A" : "g", "B": "s", "Q" : "s", "A1": "s"}]

# df1 = pd.DataFrame(df1)
# df2 = pd.DataFrame(df2)
# df3 = pd.DataFrame(df3)

# new_df = df1.assign(key=1).merge(df2.assign(key=1),
#                                  on='key').drop('key', axis=1)

# new_df = pd.concat([df3, new_df])
# print(new_df)


    A    B                                                 Q  \
0   g    s                                                 s   
0  OK  OK2  What investigation was conducted in well 15/5-1?   
1  OK  OK2                                              What   

                          A1  
0                          s  
0  Investigation of the logs  
1              Investigation  


In [113]:

# df = pd.DataFrame()
# error_list = []
# for list_dict in responses:
#     try:
#         reformed = ast.literal_eval(list_dict)
#         df1 = pd.DataFrame(reformed)
#         df = pd.concat([df, df1])
#     except:
#         print("Not proper output")
#         error_list.append(list_dict)

# df.to_csv("QA_completed", sep=',', index=False, encoding='utf-8')

Not proper output


In [56]:
import asyncio
from openai import AsyncAzureOpenAI
import time

number_concurrencies = 30
error_list = []

client = AsyncAzureOpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"]
)

sem = asyncio.Semaphore(number_concurrencies)

async def qna_model(data):
    
    prompt = data["prompt"]
    passage_id = data["Passage_id"]
    # print(prompt)
    print(passage_id)
    try:
        response = await client.chat.completions.create(
            model=os.environ["GPT432k_DEPLOYMENT"], 
            messages=[
                {"role": "system", "content": system_prompt}, 
                {"role": "user", "content": prompt}
            ])
        # print(response)
        response = response.choices[0].message.content
        # response = f'''{response}'''
        
        # Do something with chat_completion
        try:
            
            
            response = ast.literal_eval(f'''{response}''')
            if response:
                for qa_pairs in response:
                    
                    qa_pairs['Passage_id'] = passage_id
            else:
                error_list.append({"Passage_id": passage_id, "Response":response})
            
            return response
        except Exception as e:
            print(e)
            print("Something failed")
            error_list.append({"Passage_id": passage_id, "Response":response})
        
    except Exception as e:  # Replace this with the specific exception for rate limit errors
       error_list.append({"Error":passage_id, "Response":"No response"})

async def n_qna_model(data):
    async with sem:
        return await(qna_model(data))

async def questions_to_answer(data_list):
    
    return await asyncio.gather(*[asyncio.ensure_future(n_qna_model(data)) for data in data_list])

responses = await questions_to_answer(data_list)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [47]:
print(error_list)

[]


In [48]:
print(responses)
test_df = pd.DataFrame()
for df_small in responses:
    test_df2 = pd.DataFrame(df_small)
    test_df = pd.concat([test_df, test_df2])

best_df = data_df.merge(test_df, on="Passage_id", how="left")

[[{'Q': 'Where are the job reports for Wellbore 25/8-10 S filed?', 'A1': "Halliburton's office in Tananger", 'A2': "The job reports for Wellbore 25/8-10 S are filed at Halliburton's office in Tananger.", 'C': '1', 'Passage_id': 1}, {'Q': 'To what depth was the 36 hole of Wellbore 25/8-10 S drilled?', 'A1': '229 MD', 'A2': 'The 36 hole of Wellbore 25/8-10 S was drilled to 229 MD.', 'C': '1', 'Passage_id': 1}, {'Q': 'What was the discussion prior to the job for Wellbore 25/8-10 S?', 'A1': 'compressive strength development of conductor cement slurries', 'A2': 'For Wellbore 25/8-10 S, the discussion prior to the job was about the compressive strength development of conductor cement slurries.', 'C': '1', 'Passage_id': 1}, {'Q': 'How many options were discussed for Wellbore 25/8-10 S?', 'A1': 'Several', 'A2': 'Several options were discussed for Wellbore 25/8-10 S.', 'C': '1', 'Passage_id': 1}, {'Q': 'What was the conduct of the job for Wellbore 25/8-10 S?', 'A1': 'conducted with single 15.80

In [53]:
timing = str(time.time())
best_df.to_csv(f"{data_link}_{timing}.csv")

In [138]:
print(best_df)

                 INFO_ITEM_TYPE                                       _id  \
0   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
1   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
2   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
3   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
4   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
5   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
6   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
7   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
8   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
9   DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
10  DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   
11  DISCOVERY_EVALUATION_REPORT  59d3b7de417e13085ff1dbee2ce826ff2fe23165   

In [134]:
error_list = []

test3 = ast.literal_eval(test2)

In [109]:
for item in test3:
    item["test"] = "my name"
print(test3[0])

{'Q': 'What does section 1.1.5 deal with according to the information about wellbore 15/5-1?', 'A1': 'Alternatives to a piled steel jacket', 'A2': 'The section 1.1.5 in the context of wellbore 15/5-1 deals with the alternatives to a piled steel jacket.', 'C': '1', 'test': 'my name'}


In [81]:
opera = [{"2": "one"}, {"3":"two"}]

for sonette in opera:
    sonette["4"] = "three"

print(opera)

[{'2': 'one', '4': 'three'}, {'3': 'two', '4': 'three'}]
